# Data Engineering Group Project

## Team: Data Magicians
Members:
<p>
Arda Putra Ryandika
</p>
<p>
Atthaya Busayaruengrat (Hong)
</p>
<p>
Jingxue (Vera) Cao
</p>
<p>
Katharina Wiedmann  
</p>
<p>
Ying Tung (Debbie) Lau
</p>

## Objective
In this project, we are taking three sources of movie review data (csv, tsv, web scraping) and aiming to create weak labelling functions based on the data. The objective of the project is to compare the performance of a machine learning based classifier with that of the combination of weak labelling functions. 


## Movie Review Data

We split the work of obtaining data to three group members (Hong, Debbie, Vera) . Hong was responsible for scraping movie reviews from the rotten tomato website, while Debbie and Vera sought for other data formats like tsv and csv to ensure the sufficiency of data sample. 

The dataset contains two columns: one is the text review posted by people, another is the label 1 or 0. 1 indicates a positive review (fresh), and 0 indicates an non-positive review (not fresh).

The final dataset consists of 15000 reviews in total, with 5000 from web scrapping, 5000 from csv file, and 5000 from tsv file. 

We further sampled 2000 data points for labelling function development (development data set) and made sure the positive and non-positive reviews had the same amounts. 


## Building classifiers 

Arda was responsible for building a NLP classifier, which will be later applied to compare with the labelling function classifier. This model was done by utilizing tokens generated from two types of reviews and fed as features. This NLP classifier yielded 70% accuracy on the testing set. 
Arda also implemented the spark on Faculty so that the following labelling functions can be implemented in a spark environment. 

## Building Labelling functions

Meanwhile, the rest of the team members worked on generating weak labelling functions based on the findings in data exploration. 

During data exploration, as implementing spark slowed down the computation process as it partitions the dataset, we chose to use pandas to notice any patterns and differences in positive and non-positive reviews. We faced a few challenges in this stage. For example, we built a lemmatization function on the word count dataframe to avoid classifier bias caused by word inflections. However, many words were converted into completely different words incorrectly. Due to the mis-correction on words, we decided not to use lemmatization. 

As for building the labelling functions, Katie looked for stop words in the review first and counted the word occurrences in positive reviews and negative reviews. By identifying the difference in the word occurrences, we built our labelling functions to separate positive and non-positive reviews based on exclusive words. 

We also looked for capital letters mentioned in each type of reviews, but since the capital letters were irrelevant to emotions and most didn’t make sense for understanding, we decided not to create a labelling function based on it.

As we noticed keywords like “too” and “far” occurred more often in a specific type of reviews, so other labelling functions were created based on the keywords. 
Similarly, exclamation mark and question mark also appeared more often in one type of reviews, so we created labelling functions based on them as well.

After finalising the labeling functions, Arda built a classifier to combine the labeling functions together. 

## Results

<>

## GitHub
Every time we made a change, we used terminal in Faculty to push the changes to our group repository. 
After committing the change, we used “git status” to double check the state of repository. Using “git diff” also enables us to see all the changes in repository.

Link to GitHub repository:
https://github.com/KatharinaWiedmann/DataEngGroupProject.git


## JIRA board 
We used JIRA to manage the progress of our project and record our meeting topics. The brief of meetings is shown as below:
<p>
First meeting:
•	Rotten Tomatoes labelling functions - good movie or not (positive/ negative rating). 
•	Twitter data: Labelling whether someone is a Boris Johnson supporter or not. 
•	YouTube comments: positive or negative comment 
•	Promotion emails - is an email a spam email or a genuine email. 
</p>
<p>
Second meeting:
•	Labelling functions (Vera, Debbie, Katie) 
•	Classifier (Arda) 
•	Web Scraping (Hong) 
•	GitHub (Katie)
</p>
<p>
Third meeting:
•	Rewriting labelling functions (SparkLFApplier - done together)
•	Combining labelling functions (Hong & Katie)
•	Analyzing summary/ results labelling functions (Hong & Katie)
•	Plug in classifier (Arda)
•	Compare results between using labelling functions and not labelling functions (Arda)
•	Iterate on Mark-up/ write-down (Vera)
•	JIRA cleanup & additional notes (Katie)
•	Github reminder - don't forget to push/ pull (everyone)
•	Make sensitivity analysis work (Debbie) 
•	nt function (Hong)
</p>

Further details can be found at:
http://csjira2.cs.ucl.ac.uk:8080/secure/RapidBoard.jspa?rapidView=316&view=detail&selectedIssue=DED-16


In [450]:
conda install pandas==0.24.2

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - pandas==0.24.2


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2019.11.~ --> pkgs/main::ca-certificates-2020.1.1-0
  openssl            conda-forge::openssl-1.1.1d-h516909a_0 --> pkgs/main::openssl-1.1.1d-h7b6447c_4

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2019.11.28-py36h~ --> pkgs/main::certifi-2019.11.28-py36_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [451]:
# conda install networkx==2.3.0
# run once and then need to restart the kernel?

In [452]:
conda upgrade --all -y

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3


The following packages will be REMOVED:

  python_abi-3.6-1_cp36m

The following packages will be UPDATED:

  networkx                                         2.3-py_0 --> 2.4-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [453]:
conda install networkx==2.3.0

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - networkx==2.3.0


The following packages will be DOWNGRADED:

  networkx                                         2.4-py_0 --> 2.3-py_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [454]:
#needs to show version 2.3

import networkx as nx
nx.__version__

'2.3'

In [455]:
conda install snorkel==0.9.0 -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - snorkel==0.9.0


The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.6-1_cp36m

The following packages will be UPDATED:

  certifi              pkgs/main::certifi-2019.11.28-py36_0 --> conda-forge::certifi-2019.11.28-py36h9f0ad1d_1

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge::ca-certificates-2019.11.28-hecc5488_0
  openssl              pkgs/main::openssl-1.1.1d-h7b6447c_4 --> conda-forge::openssl-1.1.1d-h516909a_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [456]:
conda install -c conda-forge textblob

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [457]:
from snorkel.labeling import LFAnalysis
from snorkel.labeling import labeling_function
from snorkel.labeling import PandasLFApplier,LabelModel
from snorkel.preprocess import preprocessor
from textblob import TextBlob
import nltk
from itertools import repeat
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests
from bs4 import BeautifulSoup
from csv import writer
import re
import pickle
import time
import json

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/faculty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [458]:
#Spark 

# Spark Environment
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
import pyspark

number_cores = 4
memory_gb = 16
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext.getOrCreate(conf=conf)
print(sc)

# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

<SparkContext master=local[4] appName=pyspark-shell>


# Web Scraping

## Browse all from DVD releases page

In [459]:
main = 'https://www.rottentomatoes.com/api/private/v2.0/browse?maxTomato=100&services=amazon%3Bhbo_go%3Bitunes%3Bnetflix_iw%3Bvudu%3Bamazon_prime%3Bfandango_now&certified&sortBy=release&type=dvd-streaming-all&page='

In [460]:
# Get movie url
movie_url = []
start_page = 1 ; end_page = 1
while start_page <= end_page:
#     time.sleep(7)
    url = main + str(start_page)
    response = requests.get(url)
    if response.status_code !=200:
        print('Request error')
        break
    file = json.loads(response.text)
    for i in file['results']:
        movie_url = movie_url + [i['url']]
    start_page +=1

In [461]:
print('Examples for the url:\n')
for i in range(3):
    print(movie_url[i])
print('\nNumber of movies in list: {}'.format(len(movie_url)))

Examples for the url:

/m/frozen_ii
/m/playmobil_the_movie
/m/charlies_angels_2019

Number of movies in list: 32


In [462]:
# Split into lists of 50 movies to do the scraping
movie_url_split = [movie_url[i:i+50] for i in range(0,600,50)]

In [463]:
len(movie_url_split)

12

In [464]:
# Get reviews from the web
reviews = []
titles = []
ratings = []
for split in movie_url_split: # Loop through each split
#     time.sleep(7)
    for title in split: # Loop through each movie title
        url = 'https://www.rottentomatoes.com'+title
#         time.sleep(7)
        response = requests.get(url)
        # Check the request status code
        if response.status_code != 200:
            print('Request error')
            break
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get labels from each review (fresh vs. rotten)
        fresh_rotten = soup.find_all(class_="review_quote")
        
        # Get movie title
        title = soup.find(class_="mop-ratings-wrap__title mop-ratings-wrap__title--top").getText()
        
        # Get reviews
        review = soup.find_all('blockquote')
        for i in review:
            j = str(i.contents[1])
            j = j.replace("<p>\n                    \n                        ","")
            j = j.replace("\n                    \n                </p>","")
            reviews = reviews + [j]
            titles = titles + [title]
        
        # Identify labels
        for i in fresh_rotten:
            temp = str(i.findChildren()[2])
            if re.search('rotten',temp):
                ratings = ratings + ['rotten']
            else:
                ratings = ratings + ['fresh']
            

In [465]:
# Create the DataFrame to store the scraped data
df = pd.DataFrame([titles,reviews,ratings],index = ['title','review','rating']).T

In [466]:
# Clean the data (drop duplicates, check missing values etc.)
df = df.drop_duplicates()
df = df.replace([None],np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 329 entries, 0 to 329
Data columns (total 3 columns):
title     329 non-null object
review    329 non-null object
rating    329 non-null object
dtypes: object(3)
memory usage: 10.3+ KB


In [467]:
df.dropna(inplace=True)
df.head(3)

,title,review,rating
0,Frozen II,Frozen II is a worthy follow-up with enough he...,fresh
1,Frozen II,[Some] sequences have a gleam and a rhythm to ...,fresh
2,Frozen II,...one of the most beautifully animated films ...,fresh


In [468]:
# Export to CSV files
# df.to_csv('web_scraping_rotten_tomatoes.csv')

# Reading & Preparing TSV file 

In [469]:
# Read TSV file
tsv_reviews = pd.read_csv('/project/reviews.tsv', sep='\t', header=0, encoding='unicode_escape')

In [470]:
tsv_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [471]:
# Extract review and fresh columns
tsv_reviews = pd.DataFrame(tsv_reviews, columns = ['review', 'fresh'])

In [472]:
tsv_reviews.head()

,review,fresh
0,A distinctly gallows take on contemporary fina...,fresh
1,It's an allegory in search of a meaning that n...,rotten
2,... life lived in a bubble in financial dealin...,fresh
3,Continuing along a line introduced in last yea...,fresh
4,... a perverse twist on neorealism...,fresh


In [473]:
tsv_reviews.isnull().sum()

review    5563
fresh        0
dtype: int64

In [474]:
# drop NaN rows in reviews
index_name = tsv_reviews[(tsv_reviews['review'].isnull())].index
tsv_reviews.drop(index_name, inplace= True)

In [475]:
tsv_reviews.isnull().sum()

review    0
fresh     0
dtype: int64

In [476]:
# rename fresh as 1 and rotten as 0
tsv_reviews['fresh'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

In [477]:
#Rename columns
tsv_reviews.rename(columns={'fresh':'Freshness','review':'Review'},inplace=True)
tsv_reviews = tsv_reviews.sample(5000)
#take 5000
tsv_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 47761 to 18651
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null object
dtypes: object(2)
memory usage: 117.2+ KB


# Reading & Preparing CSV file 

In [478]:
# Read CSV file
csv_reviews= pd.read_csv('/project/rotten_tomatoes_reviews.csv')
csv_reviews.head()


,Freshness,Review
0,1,"Manakamana doesn't answer any questions, yet ..."
1,1,Wilfully offensive and powered by a chest-thu...
2,0,It would be difficult to imagine material mor...
3,0,Despite the gusto its star brings to the role...
4,0,If there was a good idea at the core of this ...


In [479]:
#Swap Freshness and Review 
columns_titles = ["Review","Freshness"]
csv_reviews=csv_reviews.reindex(columns=columns_titles)
csv_reviews = csv_reviews.sample(5000)

csv_reviews.head()
csv_reviews.info()

,Review,Freshness
463116,Além de contar com as estupendas atuações de ...,1
32549,"Seriously undermined by the skeletal script, ...",0
152364,"Pretty to look at, the film is not quite the ...",1
444218,It's a tough job but someone's got to do it i...,1
337897,Stewart plays Cole with her million-dollar ha...,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 463116 to 201237
Data columns (total 2 columns):
Review       5000 non-null object
Freshness    5000 non-null int64
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


# Web Scraping & Preparing scrapped data 

In [480]:
#Read Web Scraping Data
web_scraping_reviews= pd.read_csv('/project/web_scraping_rotten_tomatoes.csv')
web_scraping_reviews.head()

web_scraping_reviews = web_scraping_reviews.sample(5000)

,Unnamed: 0,title,review,rating
0,0,My Hindu Friend (Meu amigo Hindu),Babenco's cinematic farewell isn't perfect by ...,fresh
1,1,My Hindu Friend (Meu amigo Hindu),This is a good film if you are looking for som...,fresh
2,2,My Hindu Friend (Meu amigo Hindu),"My Hindu Friend is a celebration of life, love...",fresh
3,3,My Hindu Friend (Meu amigo Hindu),I wouldn't miss it; it's a film that's more th...,fresh
4,4,My Hindu Friend (Meu amigo Hindu),"...surreal, reflective (though never sentiment...",fresh


In [481]:
# rename fresh as 1 and rotten as 0
web_scraping_reviews['rating'].replace({'fresh':'1', 'rotten':'0'}, inplace = True)

#Rename Rating to Review 
web_scraping_reviews.rename(columns={'rating':'Freshness', 'review':'Review'},inplace=True)
web_scraping_reviews.head()

# Extract Review and Freshness columns
web_scraping_reviews= pd.DataFrame(web_scraping_reviews, columns = ['Review', 'Freshness'])

,Unnamed: 0,title,Review,Freshness
3553,3553,Brightburn,Brightburn takes a cool premise and executes i...,1
2566,2566,Corporate Animals,Tries to coast on grisly slapstick and the kin...,0
3787,3787,The Intruder,It's fun! Is it brilliant or groundbreaking? No.,0
1784,1784,Eli,"Despite some good scares, Eli winds up being m...",0
4455,4455,"Hale County This Morning, This Evening",Much of the imagery is exquisite.,1


In [482]:
web_scraping_reviews.head()

,Review,Freshness
3553,Brightburn takes a cool premise and executes i...,1
2566,Tries to coast on grisly slapstick and the kin...,0
3787,It's fun! Is it brilliant or groundbreaking? No.,0
1784,"Despite some good scares, Eli winds up being m...",0
4455,Much of the imagery is exquisite.,1


# Combining all the data together 

In [483]:
# csv_reviews.info()
# tsv_reviews.info()
# web_scraping_reviews.info()



# Concat two files into all_reviews
all_reviews=pd.concat([csv_reviews, tsv_reviews,web_scraping_reviews],axis=0, sort=False)
all_reviews.head()

,Review,Freshness
463116,Além de contar com as estupendas atuações de ...,1
32549,"Seriously undermined by the skeletal script, ...",0
152364,"Pretty to look at, the film is not quite the ...",1
444218,It's a tough job but someone's got to do it i...,1
337897,Stewart plays Cole with her million-dollar ha...,1


In [484]:
all_reviews.shape

(15000, 2)

# Split into test and training set 

In [485]:
from sklearn.model_selection import train_test_split

In [486]:
all_reviews['Freshness'] = all_reviews['Freshness'].astype(int)

In [487]:
train, test = train_test_split(all_reviews,test_size=0.2,stratify = all_reviews['Freshness'])

In [488]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 301645 to 3457
Data columns (total 2 columns):
Review       12000 non-null object
Freshness    12000 non-null int64
dtypes: int64(1), object(1)
memory usage: 281.2+ KB


In [489]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 695 to 122687
Data columns (total 2 columns):
Review       3000 non-null object
Freshness    3000 non-null int64
dtypes: int64(1), object(1)
memory usage: 70.3+ KB


In [490]:
train.head()

,Review,Freshness
301645,Smokin Aces is a Viagra suppository for compu...,0
32176,"I can't call it a success, but it is certainly...",1
22258,"It's simply so cute, it's hard to resist and t...",1
1068,Killerman is the story of how an antihero is c...,0
23072,All this doped-up ennui eventually proves wear...,0


In [491]:
#get rid off training labels 
train = train.drop('Freshness', 1)

In [492]:
test['Freshness'].value_counts()

1    1771
0    1229
Name: Freshness, dtype: int64

In [493]:
#From labelled test set, extract a sample to find out about which labelling functions could be written
#Not sure how big the development split_ can be --> take sample of 1000 data points 

development_split = test.sample(2000,random_state=42)
development_split.head()


,Review,Freshness
24699,No one can dismiss 16 Blocks as a mere formul...,0
101123,"It is not as funny as it could be, and none o...",0
11913,"It's a charming, fanciful comedy in the vein o...",1
383223,Oscar's material in sight. Forgettable. One h...,0
117188,A mediocre production that nevertheless will ...,0


In [494]:
# test2000.index

In [495]:
# test1000 = pd.merge(test,test2000,how='left',on = 'Review')
# test1000 = test1000[test1000.Freshness_y.isnull()]

In [496]:
# test1000.shape

In [497]:
# test1000.drop('Freshness_y',axis=1, inplace=True)
# test1000.columns = ['Review','Freshness']
# test1000

In [498]:
# test1000.to_csv('1000_labels.csv')

In [499]:
#For finding labelling functions: 
development_split

,Review,Freshness
24699,No one can dismiss 16 Blocks as a mere formul...,0
101123,"It is not as funny as it could be, and none o...",0
11913,"It's a charming, fanciful comedy in the vein o...",1
383223,Oscar's material in sight. Forgettable. One h...,0
117188,A mediocre production that nevertheless will ...,0
687,"Roy Cohn is dead, but as Where's My Roy Cohn? ...",1
192608,Reilly falls into Ollie with impeccable preci...,1
423601,"The Hulk is a seriously repressed movie, and ...",0
45010,I Origins suggests that Cahill's sympathies ar...,0
129253,Properly ghoulish fact-based anti-abortion sn...,0


In [500]:
development_split.count()

Review       2000
Freshness    2000
dtype: int64

In [501]:
development_split.to_csv('development_split.csv')

In [502]:
# development_split = pd.read_csv('development_split.csv',index_col = 'Unnamed: 0')

In [503]:
development_split.count()

Review       2000
Freshness    2000
dtype: int64

In [504]:
development_split.head()

,Review,Freshness
24699,No one can dismiss 16 Blocks as a mere formul...,0
101123,"It is not as funny as it could be, and none o...",0
11913,"It's a charming, fanciful comedy in the vein o...",1
383223,Oscar's material in sight. Forgettable. One h...,0
117188,A mediocre production that nevertheless will ...,0


In [505]:
#Might have to get rid off index?

development_split[development_split['Freshness'] !=1].count()

Review       830
Freshness    830
dtype: int64

In [506]:
development_split['Freshness'].value_counts()

1    1170
0     830
Name: Freshness, dtype: int64

## Split into positive and negative reviews

In [507]:
development_split = pd.read_csv('development_split.csv')

In [508]:
development_split_fresh = development_split[development_split['Freshness'] == 1]
development_split_rotten = development_split[development_split['Freshness'] == 0]
development_split_fresh.head()
development_split_rotten.head()

,Unnamed: 0,Review,Freshness
2,11913,"It's a charming, fanciful comedy in the vein o...",1
5,687,"Roy Cohn is dead, but as Where's My Roy Cohn? ...",1
6,192608,Reilly falls into Ollie with impeccable preci...,1
10,35797,As good as things get in Hollywood.,1
13,397784,I really have no idea if the premise of the f...,1


,Unnamed: 0,Review,Freshness
0,24699,No one can dismiss 16 Blocks as a mere formul...,0
1,101123,"It is not as funny as it could be, and none o...",0
3,383223,Oscar's material in sight. Forgettable. One h...,0
4,117188,A mediocre production that nevertheless will ...,0
7,423601,"The Hulk is a seriously repressed movie, and ...",0


In [509]:
# fresh reviews 
development_split_fresh.info()
# rotten reviews 
development_split_rotten.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1170 entries, 2 to 1998
Data columns (total 3 columns):
Unnamed: 0    1170 non-null int64
Review        1170 non-null object
Freshness     1170 non-null int64
dtypes: int64(2), object(1)
memory usage: 36.6+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 830 entries, 0 to 1999
Data columns (total 3 columns):
Unnamed: 0    830 non-null int64
Review        830 non-null object
Freshness     830 non-null int64
dtypes: int64(2), object(1)
memory usage: 25.9+ KB


## Labelling Functions

## 1. Word occurrences

### Positive Reviews

In [510]:
# Removing punctuation 
def remove_punctuation(dataframe):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in dataframe.Review.str.lower():
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

In [511]:
#Remove punctuation from fresh reviews & turn into Series
split_fresh= pd.Series(remove_punctuation(development_split_fresh))
split_fresh.head()

0    its a charming fanciful comedy in the vein of ...
1    roy cohn is dead but as wheres my roy cohn mak...
2     reilly falls into ollie with impeccable preci...
3                   as good as things get in hollywood
4     i really have no idea if the premise of the f...
dtype: object

In [512]:
stopWordList = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves",\
                "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",\
                "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was",\
                "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and",\
                "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between",\
                "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off",\
                "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any",\
                "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than",\
                "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [513]:
#Removing stopwords from fresh

replacements = dict(zip((fr'\b{word}\b' for word in stopWordList), repeat("")))
split_fresh.replace(replacements, regex=True, inplace=True)
split_fresh.replace({r' +': ' ', r' +\.': '.'}, regex=True, inplace=True)

### Implement lemmatization

In [514]:
# Lemmatization and Count again
def stem_recount(df):
    import pandas as pd
    # Lemmatization
    from nltk import LancasterStemmer
    st = LancasterStemmer()
    newdf = df.copy()
    for i in range(0,len(df)):
        newdf.iloc[i,0] = st.stem(str(df.iloc[i,0])) 
        # Plz make sure the word column is the first column in df when using this function
    
    # Recount
    duplicate = newdf[newdf.duplicated(['index'])]
    # Plz make sure the 'index' is the column name consisting of words
    for i in range(0,len(newdf)):
        if i >= len(duplicate):
            break
        if newdf.iloc[i,0] == duplicate.iloc[i,0]:
            newdf.iloc[i,1] = newdf.iloc[i,1] + duplicate.iloc[i,1]
    return newdf

In [515]:
common_words_fresh = split_fresh.str.split(expand=True).stack().value_counts()
common_words_fresh_df = pd.DataFrame(common_words_fresh)
common_words_fresh_df = common_words_fresh_df.rename({0:'Occurence good review'}, axis='columns')
new_common_words_fresh_df = common_words_fresh_df.reset_index()

In [516]:
stem_recount(new_common_words_fresh_df)

,index,Occurence good review
0,film,251
1,movy,143
2,on,94
3,lik,85
4,story,72
5,ful,60
6,ev,56
7,film,47
8,review,47
9,way,46


In [517]:
#Get most common words in positive reviews 
top_common_words_fresh = common_words_fresh_df[common_words_fresh_df['Occurence good review'] >=4]
top_common_words_fresh

,Occurence good review
film,204
movie,111
one,94
like,85
story,72
full,60
even,56
films,47
review,47
way,46


### *** EXPLAIN WHY WE DONT USE LEMMATIZATION

<div class="alert alert-success">
<b> Reason for not using Lemmatization </b>

<p>
    Before counting the occurance of words in the movie review, we noticed that inflections in words may result in different occurances and thus generating bias during counting. For example, "enjoy" and "enjoyed" share the same root but would be counted separately if not using Lemmatization.
    </p> 
    
<p>
    The function "stem_recount" takes the root of a word and recounts the occurences. However, it posed a disadvantage of mis-normalizing words into other completely different words. For example, "movie" was identified as "movy", and "like" was identified as "lik". We thought this disadvantage exceeds the benefits of correcting word inflection, so we decided to not implement it.
    </p> 

</div>

### Negative reviews

In [518]:
#Remove punctuation from rotten & turn into Series
split_rotten= pd.Series(remove_punctuation(development_split_rotten))
split_rotten.head()

0     no one can dismiss 16 blocks as a mere formul...
1     it is not as funny as it could be and none of...
2     oscars material in sight forgettable one hit ...
3     a mediocre production that nevertheless will ...
4     the hulk is a seriously repressed movie and l...
dtype: object

In [519]:
# Removing stopwords from negative reviews
replacements = dict(zip((fr'\b{word}\b' for word in stopWordList), repeat("")))
split_rotten.replace(replacements, regex=True, inplace=True)
split_rotten.replace({r' +': ' ', r' +\.': '.'}, regex=True, inplace=True)

In [520]:
#Get most common words in negative reviews 

common_words_rotten = split_rotten.str.split(expand=True).stack().value_counts()
common_words_rotten_df = pd.DataFrame(common_words_rotten)
common_words_rotten_df = common_words_rotten_df.rename({0:'Occurence bad review'}, axis='columns')
top_common_words_rotten = common_words_rotten_df[common_words_rotten_df['Occurence bad review'] >=4]

In [521]:
top_common_words_rotten.head()

,Occurence bad review
movie,112
film,96
like,65
one,65
much,47


## Comparison of good and bad reviews

<div class="alert alert-success">
We want to find out which of the words in the good list only appear in the good movies (and not in the bad movies), vice versa and base labeling functions on these findings. We first ened to prepare the data accordingly, before we can write the labelling functions.
    <div>

In [522]:
top_fresh_words_exclusive = top_common_words_fresh.merge(top_common_words_rotten, indicator='i', how='outer', left_index=True,\
                                                         right_index=True).query('i == "left_only"').drop('i', 1)

top_rotten_words_exclusive = top_common_words_rotten.merge(top_common_words_fresh, indicator='i', how='outer', left_index=True,\
                                                           right_index=True).query('i == "left_only"').drop('i', 1)

In [523]:
top_fresh_words_exclusive.head()

,Occurence good review,Occurence bad review
2019,5.0,NaN
ability,5.0,NaN
able,6.0,NaN
absolutely,4.0,NaN
absorbing,5.0,NaN


In [524]:
top_rotten_words_exclusive.head()

,Occurence bad review,Occurence good review
angry,4.0,NaN
appeal,5.0,NaN
attempt,9.0,NaN
attempts,4.0,NaN
awful,4.0,NaN


In [525]:
#Get only positive words 
top_fresh_words_exclusive_list = top_fresh_words_exclusive['Occurence good review'].index.tolist()
top_fresh_words_exclusive_list

['2019',
 'ability',
 'able',
 'absolutely',
 'absorbing',
 'achievement',
 'acted',
 'acting',
 'actual',
 'admittedly',
 'adults',
 'adventure',
 'affair',
 'age',
 'allows',
 'alone',
 'already',
 'always',
 'amazing',
 'america',
 'among',
 'amp',
 'animation',
 'approach',
 'art',
 'artist',
 'artistic',
 'assured',
 'audacious',
 'authentic',
 'authenticity',
 'average',
 'baby',
 'beautiful',
 'beautifully',
 'beauty',
 'biggest',
 'bleak',
 'bonkers',
 'book',
 'bright',
 'brilliant',
 'brilliantly',
 'bring',
 'brings',
 'budget',
 'cameron',
 'cannot',
 'captures',
 'cartoon',
 'center',
 'change',
 'charismatic',
 'charming',
 'chemistry',
 'christmas',
 'city',
 'class',
 'clearly',
 'cleverly',
 'close',
 'cohn',
 'color',
 'combination',
 'comedies',
 'comic',
 'committed',
 'community',
 'comparison',
 'complex',
 'concept',
 'constructed',
 'contemporary',
 'continues',
 'control',
 'conventional',
 'country',
 'couple',
 'craft',
 'crafted',
 'create',
 'creating',
 'c

In [526]:
#take out the ones that seem to make sense: 
top_fresh_words_exclusive = ['absolutely',
 'addition',
 'adventure',
 'affectionate',
 'amazing',
 'ambition',
 'art',
 'artist',
 'arts',
 'atmosphere',
 'attractive',
 'awards',
 'balance',
 'beautiful',
 'beautifully',
 'beauty',
 'bond',
 'bright',
 'brilliant',
 'captivating',
 'captures',
 'celebration',
 'charm',
 'charming',
 'christmas',
 'classic',
 'clever',
 'committed',
 'consistently',
 'contemporary',
 'conventional',
 'convincingly',
 'creates',
 'creating',
 'crowdpleaser',
 'cult',
 'decade',
 'decades',
 'deep',
 'deeper',
 'deeply',
 'definitely',
 'delightful',
 'delightfully',
 'depth',
 'deserves',
 'design',
 'details',
 'different',
 'diverse',
 'dramatic',
 'early',
 'elegant',
 'emotionally',
 'engaging',
 'enjoyable',
 'enjoyed',
 'equal',
 'especially',
 'exploration',
 'extraordinary',
 'extremely',
 'familiar',
 'famous',
 'fan',
 'fantastic',
 'fantasy',
 'fascinating',
 'felt',
 'filled',
 'finest',
 'frank',
 'fresh',
 'friends',
 'friendship',
 'gags',
 'gorgeous',
 'grand',
 'happy',
 'heart',
 'hilarious',
 'honest',
 'hope',
 'huge',
 'impact',
 'insightful',
 'inspiring',
 'intelligent',
 'intense',
 'intrigue',
 'joy',
 'laugh',
 'loved',
 'mature',
 'mind',
 'mystery',
 'nostalgia',
 'novel',
 'opening',
 'passion',
 'perfect',
 'performers',
 'personal',
 'pleasure',
 'poignant',
 'power',
 'powerful',
 'precisely',
 'profound',
 'project',
 'proves',
 'provide',
 'provocative',
 'psychological',
 'quality',
 'remarkable',
 'reveals',
 'rich',
 'riveting',
 'satisfying',
 'sharp',
 'simple',
 'smart',
 'smile',
 'stunning',
 'succeeds',
 'supernatural',
 'surprise',
 'surprises',
 'surprising',
 'surprisingly',
 'sweet',
 'talents',
 'thoughtful',
 'thrills',
 'touch',
 'touching',
 'tragedy',
 'tragic',
 'tribute',
 'unique',
 'universal',
 'warm',
 'watchable',
 'welcome',
 'wit',
 'witty',
 'wonderful',
 'worthwhile',
 'worthy']

top_fresh_words_exclusive

['absolutely',
 'addition',
 'adventure',
 'affectionate',
 'amazing',
 'ambition',
 'art',
 'artist',
 'arts',
 'atmosphere',
 'attractive',
 'awards',
 'balance',
 'beautiful',
 'beautifully',
 'beauty',
 'bond',
 'bright',
 'brilliant',
 'captivating',
 'captures',
 'celebration',
 'charm',
 'charming',
 'christmas',
 'classic',
 'clever',
 'committed',
 'consistently',
 'contemporary',
 'conventional',
 'convincingly',
 'creates',
 'creating',
 'crowdpleaser',
 'cult',
 'decade',
 'decades',
 'deep',
 'deeper',
 'deeply',
 'definitely',
 'delightful',
 'delightfully',
 'depth',
 'deserves',
 'design',
 'details',
 'different',
 'diverse',
 'dramatic',
 'early',
 'elegant',
 'emotionally',
 'engaging',
 'enjoyable',
 'enjoyed',
 'equal',
 'especially',
 'exploration',
 'extraordinary',
 'extremely',
 'familiar',
 'famous',
 'fan',
 'fantastic',
 'fantasy',
 'fascinating',
 'felt',
 'filled',
 'finest',
 'frank',
 'fresh',
 'friends',
 'friendship',
 'gags',
 'gorgeous',
 'grand',
 '

In [527]:
#Get only negative words 
top_rotten_words_exclusive_list = top_rotten_words_exclusive['Occurence bad review'].index.tolist()
top_rotten_words_exclusive_list

['angry',
 'appeal',
 'attempt',
 'attempts',
 'awful',
 'badly',
 'balance',
 'barely',
 'basic',
 'beats',
 'believe',
 'beyond',
 'birds',
 'blah',
 'bland',
 'boring',
 'brothers',
 'bunch',
 'cage',
 'camera',
 'central',
 'chase',
 'cheap',
 'clichés',
 'core',
 'crazy',
 'crime',
 'del',
 'demands',
 'derivative',
 'designed',
 'didnt',
 'die',
 'disaster',
 'dog',
 'dull',
 'either',
 'entire',
 'episode',
 'es',
 'expected',
 'failure',
 'faith',
 'falls',
 'fear',
 'flat',
 'flick',
 'forgettable',
 'four',
 'gags',
 'generic',
 'girls',
 'given',
 'god',
 'gone',
 'halfbaked',
 'hall',
 'hannibal',
 'havent',
 'help',
 'hits',
 'hold',
 'hoped',
 'huge',
 'indeed',
 'insight',
 'instead',
 'intentions',
 'interested',
 'involved',
 'john',
 'jump',
 'la',
 'lead',
 'leave',
 'let',
 'lines',
 'lowbrow',
 'mean',
 'meant',
 'melodrama',
 'misses',
 'mob',
 'moment',
 'momentum',
 'moving',
 'mr',
 'multiple',
 'nearly',
 'necessarily',
 'needs',
 'neither',
 'next',
 'number'

In [528]:
#take out the ones that seem to make sense: 
top_rotten_words_exclusive = [
 'attempt',
 'awkward',
 'barely',
 'basically',
 'bizarre',
 'bland',
 'boring',
 'clumsy',
 'comedic',
 'disappointing',
 'disappointingly',
 'disappointment',
 'disaster',
 'dull',
 'effort',
 'failed',
 'fails',
 'generic',
 'irritating',
 'lacking',
 'manic',
 'missing',
 'nobody',
 'noir',
 'none',
 'painfully',
 'pointless',
 'poorly',
 'problem',
 'shallow',
 'shame',
 'sloppy',
 'slow',
 'suffers',
 'superficial',
 'try',
 'unfortunately',
 'unfunny',
 'worst']
top_rotten_words_exclusive

['attempt',
 'awkward',
 'barely',
 'basically',
 'bizarre',
 'bland',
 'boring',
 'clumsy',
 'comedic',
 'disappointing',
 'disappointingly',
 'disappointment',
 'disaster',
 'dull',
 'effort',
 'failed',
 'fails',
 'generic',
 'irritating',
 'lacking',
 'manic',
 'missing',
 'nobody',
 'noir',
 'none',
 'painfully',
 'pointless',
 'poorly',
 'problem',
 'shallow',
 'shame',
 'sloppy',
 'slow',
 'suffers',
 'superficial',
 'try',
 'unfortunately',
 'unfunny',
 'worst']

## Labelling Function

### 1. Word Occurences

###  A. Good / bad exclusive words occurrences

In [529]:
from snorkel.labeling.apply.spark import SparkLFApplier

from pyspark import SparkContext 
from pyspark.sql import SQLContext 
import pandas as pd 
sqlc=SQLContext(sc) 
df=pd.read_csv('/project/development_split.csv',index_col = 'Unnamed: 0')
df_with_punctuation = df.copy()
df['Review'] = remove_punctuation(df)
development_split=sqlc.createDataFrame(df)
development_split_with_punctuation=sqlc.createDataFrame(df_with_punctuation) 

In [530]:
development_split.show(5)

+--------------------+---------+
|              Review|Freshness|
+--------------------+---------+
| no one can dismi...|        0|
| it is not as fun...|        0|
|its a charming fa...|        1|
| oscars material ...|        0|
| a mediocre produ...|        0|
+--------------------+---------+
only showing top 5 rows



In [531]:
# development_split = pd.read_csv('/project/development_split.csv')
ABSTAIN = -1
NOTFRESH = 0
FRESH = 1

@labeling_function()
def fresh(x):
    for word in top_fresh_words_exclusive:
        word = " " +word+" "
        if word in str(x).lower():
            return FRESH
    return ABSTAIN
#return FRESH if "best" in x.str.lower() else ABSTAIN

@labeling_function()
def rotten(x):
    for word in top_rotten_words_exclusive:
        word = " " +word+" "
        if word in str(x).lower():
            return NOTFRESH
    return ABSTAIN
#return NOTFRESH if "best" in x.str.lower() else ABSTAIN

In [532]:
lfs = [fresh]
applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [533]:
sample_L

array([[-1],
       [ 1],
       [ 1],
       ...,
       [ 1],
       [ 1],
       [ 1]])

In [534]:
coverage_fresh = (sample_L != ABSTAIN).mean(axis=0)
print("fresh coverage:{:.1%}".format(coverage_fresh[0]))

fresh coverage:35.0%


In [535]:
lfs = [rotten]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [536]:
coverage_rotten = (sample_L != ABSTAIN).mean(axis=0)
print("rotten coverage:{:.1%}".format(coverage_rotten[0]))

rotten coverage:9.4%


### B. Word 'too' occurances

In [537]:
common_words_fresh_df[common_words_fresh_df.index == 'too']

,Occurence good review
too,26


In [538]:
common_words_rotten_df[common_words_rotten_df.index == 'too']

,Occurence bad review
too,47


In [539]:
@labeling_function()
def keyword_too(x):
    return NOTFRESH if 'too' in str(x).lower() else ABSTAIN

In [540]:
lfs = [keyword_too]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [541]:
coverage_keyword_too = (sample_L != ABSTAIN).mean(axis=0)
print("keyword too coverage:{:.1%}".format(coverage_keyword_too[0]))

keyword too coverage:4.5%


### C. Word 'far' occurrences

In [542]:
common_words_fresh_df[common_words_fresh_df.index == 'far']

,Occurence good review
far,9


In [543]:
common_words_rotten_df[common_words_rotten_df.index == 'far']

,Occurence bad review
far,12


In [544]:
@labeling_function()
def keyword_far(x):
    return FRESH if 'far' in str(x).lower() else ABSTAIN

In [545]:
lfs = [keyword_far]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [546]:
coverage_keyword_far = (sample_L != ABSTAIN).mean(axis=0)
print("keyword far coverage:{:.1%}".format(coverage_keyword_far[0]))

keyword far coverage:1.8%


### D. "n't" words occurrences

In [547]:
# Exploration on the n't
# Word occurancy that with punctuation with it

# Word occurrences dataframe for fresh reviews
development_split_fresh_1 = split_fresh.str.split(expand=True).stack().value_counts()
development_split_fresh_df = pd.DataFrame(development_split_fresh_1).reset_index()

# Words occurrences dataframe for rotten reviews
development_split_rotten_1 = split_rotten.str.split(expand=True).stack().value_counts()
development_split_rotten_df = pd.DataFrame(development_split_rotten_1).reset_index()

In [548]:
@labeling_function()

def t(x):
    if re.search("'t",str(x).lower()):
        return NOTFRESH
    return ABSTAIN
#return FRESH if "best" in x.str.lower() else ABSTAIN

In [549]:
lfs = [t]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [550]:
coverage_t = (sample_L != ABSTAIN).mean(axis=0)
print("keyword far coverage:{:.1%}".format(coverage_t[0]))

keyword far coverage:13.7%


### E. Occurenes of good & bad words from external list 

<div class="alert alert-success">
We also want to look at an imported list of postive and negative words and see whether we can base the labelling functions on them.
    </div>


In [551]:
#Importing good and bad words & preparing for labelling function 

In [552]:
#POSITIVE WORDS 
#positive words from --> DON't DELETE! NEED TO CITE PROPERLY http://ptrckprry.com/course/ssd/data/positive-words.txt
positive_word = pd.read_csv('/project/positive_words.csv')

#sample 500 words 
positive_word = positive_word.sample(500)

#convert it into a list 
positive_word= positive_word['a+'].tolist()



#NEGATIVE WORDS 
#negative words from --> HONG?? 
negative_word = pd.read_csv('/project/negative_words.csv')

#sample 500 words 
negative_word = negative_word.sample(500)

#convert it into a list 
negative_word= negative_word['2-faces'].tolist()


In [553]:
@labeling_function()
def negative(x): 
    for word in negative_word:
        word = " " + word + " "
        if word in str(x).lower():
            return NOTFRESH 
    return ABSTAIN 


@labeling_function()
def positive(x):
    for word in positive_word:
        word = " " + word + " "
        if word in str(x).lower():
            return FRESH 
    return ABSTAIN 

In [554]:
lfs = [negative]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [555]:
coverage_negative = (sample_L != ABSTAIN).mean(axis=0)
print("negative words coverage:{:.1%}".format(coverage_negative[0]))

negative words coverage:10.8%


In [556]:
lfs = [positive]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [557]:
coverage_positive = (sample_L != ABSTAIN).mean(axis=0)
print("positive words coverage:{:.1%}".format(coverage_positive[0]))

positive words coverage:22.8%


## 2. Punctuation occurrences

In [558]:
# Turn review column into Series
development_split_fresh_series = pd.Series(development_split_fresh.Review)
development_split_rotten_series = pd.Series(development_split_rotten.Review)

In [559]:
# Positive reviews
# Split reviews into word
fresh_split = pd.Series(development_split_fresh_series.str.split(expand=True).stack())
fresh_words = [i for i in fresh_split]

# Split words into characters
def split_str():
    return [list(ch) for ch in fresh_words]
fresh_split_words = pd.Series(split_str())

In [560]:
# Negative reviews
# Split reviews into word
rotten_split = pd.Series(development_split_rotten_series.str.split(expand=True).stack())
rotten_words = [i for i in rotten_split]

# Split words into characters
def split_str():
    return [list(ch) for ch in rotten_words]
rotten_split_words = pd.Series(split_str())

In [561]:
# Turn into a flattened list
fresh_flattened_list = [y for x in fresh_split_words for y in x]
rotten_flattened_list = [y for x in rotten_split_words for y in x]

# Count the occurancy of each character
# Positive reviews
fresh_split_characters = pd.Series(fresh_flattened_list).value_counts()
fresh_split_characters = pd.DataFrame(fresh_split_characters).reset_index()

# Negative reviews
rotten_split_characters = pd.Series(rotten_flattened_list).value_counts()
rotten_split_characters = pd.DataFrame(rotten_split_characters).reset_index()

### A. Question mark occurrences

In [562]:
# Count the # of occurance of '?' in fresh reviews
fresh_split_characters[fresh_split_characters['index'] == '?']

,index,0
61,?,22


In [563]:
# Count the # of occurance of '?' in rotten reviews
rotten_split_characters[rotten_split_characters['index'] == '?']

,index,0
53,?,28


In [564]:
list_with_question_mark = []
for review in development_split_rotten.Review:
    if '?' in review:
        list_with_question_mark.append(review)
        
print (list_with_question_mark[:3])

['Prides itself on a clever twist and a few great red herrings. You know how I know? Because it actually tells us. Literally. As in, in the dialogue.', ' At some point you have to start asking with all the mystical potions and gifts of foresight, how do these wizards keep fingering the wrong people for these crimes?', " Poor Guido. Poor Marshall. Poor audiences, who likely were eager to see this movie (given the hype, who wasn't?), but who likely will leave it knowing they deserved better."]


In [565]:
@labeling_function()
def question_mark(x):
    return NOTFRESH if '?' in str(x).lower() else ABSTAIN

In [566]:
lfs = [question_mark]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split_with_punctuation.rdd)

In [567]:
coverage_question_mark = (sample_L != ABSTAIN).mean(axis=0)
print("question mark coverage:{:.1%}".format(coverage_question_mark[0]))

question mark coverage:2.2%


### B. Exclamation mark occurrences

In [568]:
# Count the # of occurance of '!' in fresh reviews
fresh_split_characters[fresh_split_characters['index'] == '!']

,index,0
65,!,13


In [569]:
# Count the # of occurance of '!' in rotten reviews
rotten_split_characters[rotten_split_characters['index'] == '!']

,index,0
61,!,17


In [570]:
@labeling_function()
def exclamation_mark(x):
    return FRESH if '!' in str(x).lower() else ABSTAIN

In [571]:
lfs = [exclamation_mark]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split_with_punctuation.rdd)

In [572]:
coverage_exclamation_mark = (sample_L != ABSTAIN).mean(axis=0)
print("exclamation mark coverage:{:.1%}".format(coverage_exclamation_mark[0]))

exclamation mark coverage:1.2%


### 3. Combining labelling functions

<div class="alert alert-success">
Next, we want to combine all the labelling functions into one and apply them to the training set. However, as the labelling functions around the punctuation have very low coverages, we decided not to include these.
    </div>

In [573]:
lfs = [fresh,
       rotten,
       keyword_too,
       keyword_far,
       t,
       negative,
       positive]

applier = SparkLFApplier(lfs)
sample_L = applier.apply(development_split.rdd)

In [574]:
train_prepared = train.copy()

train_prepared
train_prepared['Review'] = remove_punctuation(train_prepared)

,Review
301645,Smokin Aces is a Viagra suppository for compu...
32176,"I can't call it a success, but it is certainly..."
22258,"It's simply so cute, it's hard to resist and t..."
1068,Killerman is the story of how an antihero is c...
23072,All this doped-up ennui eventually proves wear...
365470,"Like Moby Dick, film is not just about market..."
4046,Alita: Battle Angel has incredibly stunning vi...
317502,"This is a leaky vessel sailing in circles, no..."
2244,"In short, CRAWL has bite. For the most part, e..."
82390,Does no one know how to film physical comedy ...


In [575]:
from snorkel.labeling import LFAnalysis
LFAnalysis(L=sample_L, lfs = lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
fresh,0,[1],0.3505,0.2050,0.1210
rotten,1,[0],0.0940,0.0565,0.0345
keyword_too,2,[0],0.0445,0.0340,0.0240
keyword_far,3,[1],0.0185,0.0130,0.0090
t,4,[0],0.1370,0.0865,0.0730
negative,5,[0],0.1080,0.0705,0.0520
positive,6,[1],0.2275,0.1580,0.0745


In [576]:
lfs = [fresh,
       rotten,
       keyword_too,
       keyword_far,
       t,
       negative,
       positive]

applier = SparkLFApplier(lfs)

L_train=sqlc.createDataFrame(train_prepared)

#is this next line correct?
L_test = sqlc.createDataFrame(test)

# type(L_train)
L_train = applier.apply(L_train.rdd)

#is this next line correct?
L_test = applier.apply(L_test.rdd)


In [577]:
type(L_train)

numpy.ndarray

In [578]:
L_train

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1,  0, -1, ..., -1, -1, -1],
       [ 1, -1, -1, ..., -1,  0,  1],
       ...,
       [ 1, -1, -1, ..., -1, -1,  1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]])

In [579]:
from snorkel.labeling import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [580]:
preds_train.shape

(12000,)

In [581]:
preds_train

array([0, 0, 1, ..., 1, 0, 0])

In [582]:
train2 = train.copy()
train2['predicted_train'] = preds_train
train2.to_csv('12000_predicted_labels.csv')

In [583]:
#needs to show version 2.3
import networkx as nx
nx.__version__

'2.3'

In [584]:
#Labelling according to weights 
from snorkel.labeling import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [585]:
#need to be checked 
L_test = L_test
Y_test = test['Freshness']

In [586]:
majority_acc = majority_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   59.4%
Label Model Accuracy:     60.0%


## Sensitivity Analysis

In [587]:
# conda install py4j

In [588]:
import py4j

In [589]:
from pyspark.context import SparkContext
from pyspark import SQLContext, SparkConf

print('Spark version:', sc.version)

print('Python version:', sys.version)

#Apparently PySpark does not work with Python 3.6.10
#This is why we get the Java4 Error

Spark version: 2.4.5
Python version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]


In [590]:
from snorkel.preprocess import preprocessor
from textblob import TextBlob

# @preprocessor(memoize=True)
# def textblob_sentiment(x):
#     scores = TextBlob(x.text)
#     x.polarity = scores.sentiment.polarity
#     x.subjectivity = scores.sentiment.subjectivity
#     return x

# #pick a reasonable threshold 
# #Using a lower threshold than other examples as this could be a good indicator for determining official sources vs general negative sentiment
# @labeling_function()
# def textblob_polarity(x):
#     return NOTFRESH if x.polarity > 0.8 else ABSTAIN

# #do the same for the subjectivity scores. 
# #Using a higher threshold than other examples as this could be a good indicator for determining official sources
# #This will run faster than the last cell, since we memoized the Preprocessor outputs.
# @labeling_function()
# def textblob_subjectivity(x):
#     return FRESH if x.subjectivity >= 0.6 else ABSTAIN

In [618]:
def getSentiment(text):
    x = {}
    x["polarity"] = TextBlob(text).sentiment.polarity
    x["subjectivity"] = TextBlob(text).sentiment.subjectivity
    return x

In [619]:
# @labeling_function(pre=[textblob_sentiment])
# def textblob_polarity(x):
#     x = getSentiment(x.text)
#     return FRESH if x.polarity > 0.8 else ABSTAIN

# @labeling_function(pre=[textblob_sentiment])
# def textblob_subjectivity(x):
#     x = getSentiment(x.text)
#     return FRESH if x.subjectivity >= 0.5 else ABSTAIN



#pick a reasonable threshold 
#Using a lower threshold than other examples as this could be a good indicator for determining official sources vs general negative sentiment
#@labeling_function(pre=[textblob_sentiment])
@labeling_function()
def textblob_polarity(x):
    x = getSentiment(x.Review)
    return FRESH if x["polarity"] > 0.8 else ABSTAIN

#do the same for the subjectivity scores. 
#Using a higher threshold than other examples as this could be a good indicator for determining official sources
#This will run faster than the last cell, since we memoized the Preprocessor outputs.
#@labeling_function(pre=[textblob_sentiment])
@labeling_function()
def textblob_subjectivity(x):
    x = getSentiment(x.Review)
    return FRESH if x["subjectivity"] >= 0.5 else ABSTAIN

In [612]:
development_split.show()

+--------------------+---------+
|              Review|Freshness|
+--------------------+---------+
| no one can dismi...|        0|
| it is not as fun...|        0|
|its a charming fa...|        1|
| oscars material ...|        0|
| a mediocre produ...|        0|
|roy cohn is dead ...|        1|
| reilly falls int...|        1|
| the hulk is a se...|        0|
|i origins suggest...|        0|
| properly ghoulis...|        0|
|as good as things...|        1|
| tonedeaf and str...|        0|
|this is a degradi...|        0|
| i really have no...|        1|
|uglydolls worst f...|        0|
|the creation of t...|        1|
|an extraordinaril...|        1|
|the film is in tu...|        1|
|annabelle comes h...|        1|
|martial arts have...|        1|
+--------------------+---------+
only showing top 20 rows



In [613]:
development_split[development_split.Review == ""].count()

0

In [616]:
development_split.select('Freshness').distinct().show(truncate=False)

development_split[development_split.Freshness == None].count()


+---------+
|Freshness|
+---------+
|0        |
|1        |
+---------+



0

In [620]:
lfs = [
       textblob_polarity,
       textblob_subjectivity
      ]

applier = SparkLFApplier(lfs)

sample_L = applier.apply(development_split.rdd)



In [622]:
LFAnalysis(sample_L, lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
textblob_polarity,0,[1],0.0105,0.004,0.0
textblob_subjectivity,1,[1],0.5500,0.004,0.0


# Classifier

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string

# Spark Environment
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

import pyspark

number_cores = 4
memory_gb = 16
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)
sc = pyspark.SparkContext.getOrCreate(conf=conf)
print(sc)

# get the context
spark = pyspark.sql.SparkSession.builder.getOrCreate()
print(spark) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
# Download files
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

!pip install langid
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import preproc as pp

# Register all the functions in Preproc with Spark Context
check_lang_udf = udf(pp.check_lang, StringType())
remove_stops_udf = udf(pp.remove_stops, StringType())
remove_features_udf = udf(pp.remove_features, StringType())
tag_and_remove_udf = udf(pp.tag_and_remove, StringType())
lemmatize_udf = udf(pp.lemmatize, StringType())
check_blanks_udf = udf(pp.check_blanks, StringType())

In [ ]:
from pyspark.sql.types import IntegerType

# Read the data (Spark)
review_df = development_split

# Rename Column
review_df = review_df.withColumnRenamed('Review','text')
review_df = review_df.withColumnRenamed('Freshness','label')
review_df = review_df.withColumnRenamed('_c0','index')

# Change data type to Integer
review_df = review_df.withColumn("label", review_df["label"].cast(IntegerType()))

# Show df information
review_df.show()
review_df.printSchema()
review_df.count()

In [ ]:
# remove stop words to reduce dimensionality
review_df = review_df.withColumn("text", remove_stops_udf(review_df["text"]))

# remove other non essential words
review_df = review_df.withColumn("text", remove_features_udf(review_df["text"]))

# tag the words remaining and keep only Nouns, Verbs and Adjectives
review_df = review_df.withColumn("text", tag_and_remove_udf(review_df["text"]))

# lemmatization of remaining words to reduce dimensionality & boost measures
review_df = review_df.withColumn("text", lemmatize_udf(review_df["text"]))

review_df.show()

In [ ]:
#Specify Training and Test data
training_df = review_df
test_df = sqlContext.read.csv('/project/1000_labels.csv', header=True)

# Rename Column
test_df = test_df.withColumnRenamed('Review','text')
test_df = test_df.withColumnRenamed('Freshness','label')
test_df = test_df.withColumnRenamed('_c0','index')

# Change data type to Integer
test_df = test_df.withColumn("label", test_df["label"].cast(IntegerType()))


test_df.show()
test_df.printSchema()
test_df.count()

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator

# Configure an ML pipeline, which consists of tree stages: tokenizer, hashingTF, and nb.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol='words', outputCol="features")
idf = IDF(minDocFreq=3, inputCol="features", outputCol="idf")
nb = NaiveBayes()
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, nb])


paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 1.0]).build()


cv = CrossValidator(estimator=pipeline, 
                    estimatorParamMaps=paramGrid, 
                    evaluator=MulticlassClassificationEvaluator(), 
                    numFolds=4)

# Error
cvModel = cv.fit(training_df)

result = cvModel.transform(test_df)
prediction_df = result.select("text", "label", "prediction")
prediction_df.show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate the Accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(result, {evaluator.metricName: "accuracy"})